<a href="https://colab.research.google.com/github/dangdang2222/LG_Aimers/blob/main/optuna_kfold_y_07_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,13): # ignore 'ID'
        rmse = mean_squared_error(np.array(gt)[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(np.array(gt)[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [4]:
def lg_nrmse_one(gt, preds):
    # Y_08의 경우는 가중치 1.2 부여. 8 이후부터는 모두 가중치 1.0
    all_nrmse = []
    rmse = mean_squared_error(np.array(gt), preds, squared=False)
    nrmse = rmse/np.mean(np.abs(np.array(gt)))
    all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse)
    return score

#얘를 쓸것!!!

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/LG_Aimer/train.csv').drop(columns="ID")
train_x = train_df.filter(regex='X') # Input : X Featrue
train_x=train_x.drop(columns=['X_04','X_23','X_47','X_48'])
train_y = train_df.filter(regex='Y') # Output : Y Feature

##Y_03


In [7]:
train_y07 = train_y['Y_07']
train_y07
train_y07=pd.DataFrame(train_y07, index= train_y.index)
train_y07

,Y_07
0,4.276
1,3.229
2,2.839
3,3.144
4,3.138
...,...
39602,3.410
39603,3.406
39604,3.215
39605,4.216


#클리핑

In [8]:
from sklearn.model_selection import train_test_split

# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x.quantile(0.01)
p99 = train_x.quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x.clip(p01, p99, axis=1)

In [9]:
# 커넥터 핀 휨 
train_x['X__21'] = (train_x['X_24'] - train_x['X_12'])
train_x['X__22'] = (train_x['X_25'] - train_x['X_12'])
train_x['X__23'] = (train_x['X_26'] - train_x['X_12'])
train_x['X__24'] = (train_x['X_27'] - train_x['X_12'])
train_x['X__25'] = (train_x['X_28'] - train_x['X_12'])
train_x['X__26'] = (train_x['X_29'] - train_x['X_12'])
#이후, RFE 상에서 혹여나 새로운 변수를 만드는데 기여한 기존 변수들(12, 24,25,26,27,28,29)가 important하게 잡힐 수 있으므로 일단은 놔둡니다.
#또한, 현재 train_x 상에서는 통과여부 변수들은 모두 drop된 상태입니다.

In [10]:
new_col = ['X_13', 'X_19', 'X_32', 'X_07', 'X_09', 'X_49', 'X_21', 'X_03', 'X_17', 'X__22', 'X_05', 'X_22', 'X__26', 'X_08', 'X_51', 'X__23', 'X_31', 'X_14', 'X_44', 'X__25', 'X_38', 'X__21', 'X_52','X_40']
print(len(new_col))
train_set = train_x[new_col]
train_set

24


,X_13,X_19,X_32,X_07,X_09,X_49,X_21,X_03,X_17,X__22,...,X_51,X__23,X_31,X_14,X_44,X__25,X_38,X__21,X_52,X_40
0,0.18,3.11,1.46,29.45,245.71,9706.030,3.06,67.47,13.52,-2.27,...,135.359219,-2.29,1.69,13.34,21.09,-2.28,-16.41,-2.29,147.837968,-17.03
1,0.18,3.04,1.45,28.73,233.61,10423.430,2.98,65.17,13.51,-2.28,...,135.979817,-2.28,1.67,13.33,21.13,-2.21,-16.06,-2.28,149.924692,-16.74
2,0.15,3.04,1.46,28.81,272.20,10948.530,3.01,64.07,13.51,-2.30,...,131.055355,-2.32,1.69,13.36,21.12,-2.26,-16.16,-2.29,146.814592,-16.76
3,0.21,3.05,1.47,28.92,255.36,15007.030,3.02,67.57,13.51,-2.28,...,133.239422,-2.30,1.68,13.30,21.09,-2.26,-16.05,-2.27,139.720132,-16.67
4,0.16,3.04,1.47,29.68,241.46,11051.030,3.00,63.57,13.50,-2.29,...,136.620022,-2.26,1.68,13.35,21.10,-2.07,-16.25,-2.26,134.853555,-16.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,0.15,3.20,1.36,30.20,298.05,56583.294,3.06,62.27,13.52,-2.25,...,130.807148,-2.28,1.60,13.37,21.19,-2.26,-16.09,-2.26,133.481737,-16.79
39603,0.13,3.15,1.37,29.21,270.67,56583.294,3.05,62.77,13.49,-2.28,...,120.158764,-2.31,1.68,13.36,21.19,-2.29,-15.70,-2.28,142.667802,-16.40
39604,0.14,3.23,1.37,29.96,198.07,9092.372,3.07,64.67,13.52,-2.29,...,136.893025,-2.30,1.61,13.38,21.19,-2.26,-16.12,-2.25,134.419328,-16.80
39605,0.16,3.18,1.36,30.30,275.52,56583.294,3.15,63.67,13.52,-2.30,...,121.495930,-2.28,1.56,13.36,21.13,-2.23,-16.36,-2.26,141.288011,-16.97


In [11]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 15.3 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 209 kB 65.1 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 147 kB 60.5 MB/s 
     |████████████████████████████████| 49 kB 6.8 MB/s 
     |████████████████████████████████| 112 kB 76.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=1d69a545ff6f1711330ebfdda0476fa5e27d3cbbf241a2caea5a4fb8845537f2
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [12]:
import optuna
import time
from optuna import Trial, visualization
from sklearn.model_selection import KFold
import xgboost as xgb


#XGB 하이퍼 파라미터들 값 지정
def objectiveXGB(trial: Trial, X, y):  # X : train_set, y = train_y02

    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 1000, 4000,500),
        'max_depth':trial.suggest_int('max_depth', 8, 13),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 20),
        'gamma':1,
        'learning_rate': trial.suggest_discrete_uniform('learning_rate',0.004,0.01,0.0005),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.05),
        'nthread' : -1,
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.5,0.55,0.6,0.65,0.7,0.75])
    }

    kf = KFold(shuffle=True, n_splits=5, random_state=42)
    loss = []
    i = 0

    # 데이터 나누기
    X, y = X.values, y.values
    for (train_index, test_index) in kf.split(X):
        since = time.time()
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 설정된 파라미터 적용
        model = xgb.XGBRegressor(**param)

        # 학습하기
        xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])

        # 평가 및 기록
        elapsed = time.time() - since
        score = lg_nrmse_one(y_test, xgb_model.predict(X_test))
        print("Fold {} | NRMSE : {:.6f}, Elapsed Time: {:.4f}".format(i, score, elapsed))
        loss.append(score)
        i += 1

    print("Best: {:.6f}, Avg:{:.6f}".format(min(loss), np.mean(loss)))

    return np.mean(loss)

0.15573291235781145

In [ ]:
from optuna.samplers import TPESampler

# MAE가 최소가 되는 방향으로 학습을 진행
# TPESampler : Sampler using TPE (Tree-structured Parzen Estimator) algorithm.

# x_train, x_valid, y_train, y_valid = train_test_split(train_x_xgb, train_y01, test_size=0.3) 
study = optuna.create_study(direction='minimize', sampler=TPESampler())

# n_trials 지정해주지 않으면, 무한 반복
study.optimize(lambda trial : objectiveXGB(trial, train_set, train_y07), n_trials = 100)

print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
print('study.best_params:', study.best_trial.value)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('study.best_params:', study.best_params)

[I 2022-08-25 15:16:15,385] A new study created in memory with name: no-name-c62f3b97-251a-463c-b6c7-c8c376f591ba


[15:16:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.153466, Elapsed Time: 106.4816
[15:18:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.155413, Elapsed Time: 105.5220
[15:19:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.154740, Elapsed Time: 105.8824
[15:21:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.155088, Elapsed Time: 105.2155
[15:23:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 15:25:11,458] Trial 0 finished with value: 0.15445896888673472 and parameters: {'n_estimators': 1500, 'max_depth': 12, 'min_child_weight': 11, 'learning_rate': 0.006500000000000001, 'colsample_bytree': 0.5, 'lambda': 0.10570351797066953, 'alpha': 0.016021343637011465, 'subsample': 0.55}. Best is trial 0 with value: 0.15445896888673472.


Fold 4 | NRMSE : 0.153588, Elapsed Time: 105.9104
Best: 0.153466, Avg:0.154459
[15:25:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.153322, Elapsed Time: 111.2212
[15:27:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.155306, Elapsed Time: 110.6293
[15:28:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.154678, Elapsed Time: 111.1474
[15:30:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.154832, Elapsed Time: 111.1477
[15:32:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 15:34:30,906] Trial 1 finished with value: 0.1543115172015599 and parameters: {'n_estimators': 1000, 'max_depth': 12, 'min_child_weight': 2, 'learning_rate': 0.008, 'colsample_bytree': 0.9, 'lambda': 5.88287138197687, 'alpha': 3.2561899370719725, 'subsample': 0.75}. Best is trial 1 with value: 0.1543115172015599.


...

Fold 4 | NRMSE : 0.153609, Elapsed Time: 173.5675
Best: 0.153609, Avg:0.154569
[17:32:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.153510, Elapsed Time: 242.8321
[17:36:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.155271, Elapsed Time: 242.5572
[17:40:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.154744, Elapsed Time: 241.8215
[17:44:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.155046, Elapsed Time: 241.6478
[17:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 17:52:53,687] Trial 10 finished with value: 0.15441570723213552 and parameters: {'n_estimators': 4000, 'max_depth': 9, 'min_child_weight': 4, 'learning_rate': 0.004, 'colsample_bytree': 0.65, 'lambda': 0.0011878523922477712, 'alpha': 0.0010404641086628886, 'subsample': 0.7}. Best is trial 2 with value: 0.15428661476063815.


Fold 4 | NRMSE : 0.153507, Elapsed Time: 243.5792
Best: 0.153507, Avg:0.154416
[17:52:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.153526, Elapsed Time: 314.0139
[17:58:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.155139, Elapsed Time: 310.3074
[18:03:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 2 | NRMSE : 0.154711, Elapsed Time: 315.9616
[18:08:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 3 | NRMSE : 0.155084, Elapsed Time: 313.5432
[18:13:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-25 18:19:19,260] Trial 11 finished with value: 0.15440823433233014 and parameters: {'n_estimators': 3500, 'max_depth': 10, 'min_child_weight': 1, 'learning_rate': 0.0055, 'colsample_bytree': 0.9, 'lambda': 3.4180726416438696, 'alpha': 8.07287561356873, 'subsample': 0.7}. Best is trial 2 with value: 0.15428661476063815.


Fold 4 | NRMSE : 0.153581, Elapsed Time: 316.6112
Best: 0.153526, Avg:0.154408
[18:19:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 0 | NRMSE : 0.153484, Elapsed Time: 148.1710
[18:21:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fold 1 | NRMSE : 0.155192, Elapsed Time: 149.6277
[18:24:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
